In [1]:
# Implement GoogleNet
# Build an inception module

In [2]:
import functools
import keras

DefaultConv = functools.partial(
    keras.layers.Conv2D, kernel_size=(1, 1), strides=(1, 1),
    padding="same", activation="relu")

class InceptionModule(keras.layers.Layer):
  def __init__(self, filters11, filters33_reduce, filters33,
               filters55_reduce, filters55, filters_pool_proj,
               use_batch_norm=True,**kwargs):
    super().__init__(**kwargs)
    self.conv11 = DefaultConv(filters=filters11)

    self.conv33_reduce = DefaultConv(filters=filters33_reduce)
    self.conv33 = DefaultConv(filters=filters33, kernel_size=(3,3))

    self.conv55_reduce = DefaultConv(filters=filters55_reduce)
    self.conv55 = DefaultConv(filters=filters55, kernel_size=(5,5))

    self.max_pool33 = keras.layers.MaxPool2D(pool_size=(3,3), strides=(1,1), padding='same')
    self.conv_pool = DefaultConv(filters=filters_pool_proj)

    self.use_batch_norm = use_batch_norm
    if use_batch_norm:
      self.batch_layer = keras.layers.BatchNormalization()

  def call(self, inputs):
    path1 = self.conv11(inputs)
    path2 = self.conv33(self.conv33_reduce(inputs))
    path3 = self.conv55(self.conv55_reduce(inputs))
    path4 = self.conv_pool(self.max_pool33(inputs))
    # print('path1', path1.shape)
    # print('path2', path2.shape)
    # print('path3', path3.shape)
    # print('path4', path4.shape)

    iModule = keras.layers.Concatenate(axis=-1)([path1, path2, path3, path4])
    # print('imodule', iModule.shape)
    logits = self.batch_layer(iModule) if self.use_batch_norm else iModule

    return logits


2025-01-06 16:15:35.739683: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-06 16:15:35.750916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736176535.764145  409118 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736176535.768106  409118 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-06 16:15:35.782085: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
DefaultMaxPool = functools.partial(
    keras.layers.MaxPool2D,
    pool_size=(3,3), strides=(2,2), padding="same")

def get_googlenet_model(input_shape, num_classes, use_batch_norm=True, **kwargs):
  model = keras.Sequential(**kwargs)
  model.add(keras.layers.Input(shape=input_shape))
  model.add(DefaultConv(filters=64, kernel_size=(7,7), strides=(2,2),  padding='same'))
  if use_batch_norm:
    model.add(keras.layers.BatchNormalization())
  model.add(DefaultMaxPool())
  model.add(DefaultConv(filters=64))
  model.add(DefaultConv(filters=192, kernel_size=(3,3)))
  if use_batch_norm:
    model.add(keras.layers.BatchNormalization())
  model.add(DefaultMaxPool())

  model.add(InceptionModule(filters11=64, filters33_reduce=96, filters33=128,
    filters55_reduce=16, filters55=32, filters_pool_proj=32,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=128, filters33_reduce=128, filters33=192,
    filters55_reduce=32, filters55=96, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(DefaultMaxPool())

  model.add(InceptionModule(filters11=192, filters33_reduce=96, filters33=208,
    filters55_reduce=16, filters55=48, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=160, filters33_reduce=112, filters33=224,
    filters55_reduce=24, filters55=64, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=128, filters33_reduce=128, filters33=256,
    filters55_reduce=24, filters55=64, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=112, filters33_reduce=144, filters33=288,
    filters55_reduce=32, filters55=64, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=256, filters33_reduce=160, filters33=320,
    filters55_reduce=32, filters55=128, filters_pool_proj=128,
    use_batch_norm=use_batch_norm))

  model.add(DefaultMaxPool())
  model.add(InceptionModule(filters11=256, filters33_reduce=160, filters33=320,
    filters55_reduce=32, filters55=128, filters_pool_proj=128,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=384, filters33_reduce=192, filters33=384,
    filters55_reduce=48, filters55=128, filters_pool_proj=128,
    use_batch_norm=use_batch_norm))
  model.add(keras.layers.GlobalAveragePooling2D())
  model.add(keras.layers.Dropout(0.4))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(units=1000, activation="relu"))
  model.add(keras.layers.Dense(units=4, activation='sigmoid'))

  return model

In [5]:
model = get_googlenet_model(input_shape=(1024,1024,3), num_classes=4)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 512, 512, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 512, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 256, 256, 64)   │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 256, 256, 192)  │       110,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 192)  │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 128, 128, 192)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_1              │ (None, 128, 128, 256)  │       164,720 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_2              │ (None, 128, 128, 480)  │       390,656 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 64, 64, 480)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_3              │ (None, 64, 64, 512)    │       378,224 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_4              │ (None, 64, 64, 512)    │       451,208 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_5              │ (None, 64, 64, 512)    │       512,152 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_6              │ (None, 64, 64, 528)    │       607,488 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_7              │ (None, 64, 64, 832)    │       871,680 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 32, 32, 832)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_8              │ (None, 32, 32, 832)    │     1,046,784 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_9              │ (None, 32, 32, 1024)   │     1,448,176 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │             

 Total params: 7,025,532 (26.80 MB)

 Trainable params: 7,014,044 (26.76 MB)

 Non-trainable params: 11,488 (44.88 KB)

In [6]:
def iou(y_true, y_pred):
    """
    Calculate IoU loss between the true and predicted bounding boxes.

    y_true and y_pred should have the shape (batch_size, 4), where each element is
    [center_x, center_y, width, height].
    """
    # Convert (center_x, center_y, width, height) to (xmin, ymin, xmax, ymax)
    true_xmin = y_true[..., 0] - 0.5 * y_true[..., 2]
    true_ymin = y_true[..., 1] - 0.5 * y_true[..., 3]
    true_xmax = y_true[..., 0] + 0.5 * y_true[..., 2]
    true_ymax = y_true[..., 1] + 0.5 * y_true[..., 3]

    pred_xmin = y_pred[..., 0] - 0.5 * y_pred[..., 2]
    pred_ymin = y_pred[..., 1] - 0.5 * y_pred[..., 3]
    pred_xmax = y_pred[..., 0] + 0.5 * y_pred[..., 2]
    pred_ymax = y_pred[..., 1] + 0.5 * y_pred[..., 3]

    # Calculate the intersection area
    inter_xmin = keras.ops.maximum(true_xmin, pred_xmin)
    inter_ymin = keras.ops.maximum(true_ymin, pred_ymin)
    inter_xmax = keras.ops.minimum(true_xmax, pred_xmax)
    inter_ymax = keras.ops.minimum(true_ymax, pred_ymax)

    inter_width = keras.ops.maximum(0.0, inter_xmax - inter_xmin)
    inter_height = keras.ops.maximum(0.0, inter_ymax - inter_ymin)
    intersection_area = inter_width * inter_height

    # Calculate the union area
    true_area = (true_xmax - true_xmin) * (true_ymax - true_ymin)
    pred_area = (pred_xmax - pred_xmin) * (pred_ymax - pred_ymin)
    union_area = true_area + pred_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    return iou



In [7]:
# Compile the model with IoU loss
# model.compile(optimizer='adam', loss=keras.losses.Huber(), metrics=[iou_loss])
model.compile(optimizer='adam', loss='mse', metrics=[iou])


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from FrameLoader import FrameLoader
from DataGeneratorFrames import DataGeneratorFrames
from DataRepository import DataRepository

repo = DataRepository()
repo.load_relativePaths_of_videos_with_framelabels()

DIM = 320
train_generator = DataGeneratorFrames(
    frameloader=FrameLoader(repo),
    train_test_val="train",
    dim=(DIM,DIM),
    batch_size=32,
)

val_generator = DataGeneratorFrames(
    frameloader=FrameLoader(repo),
    train_test_val="test",
    dim=(DIM,DIM),
    batch_size=32,
)


callbacks = [
    ModelCheckpoint('model_best.keras', save_best_only=True, monitor='loss', mode='min', verbose=1),
    EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=1)
]

# Fit the model using the data generator and callbacks
history = model.fit(
    train_generator,  # Training data generator
    epochs=5,        # Set the number of epochs
    callbacks=callbacks,  # List of callbacks
    verbose=1,         # Verbose output
    validation_data=val_generator
)

connection established
connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
on_epoch_end_called
Epoch 1/5


I0000 00:00:1736176575.196860  409233 service.cc:148] XLA service 0x7b61900045e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736176575.196885  409233 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-01-06 16:16:15.587063: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-06 16:16:19.573937: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_7413', 236 bytes spill stores, 236 bytes spill loads

2025-01-06 16:16:20.716989: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be p

In [9]:
X, y = train_generator.__getitem__(5)
X.shape, y.shape

((0,), (0,))